# Subject-Driven Generation Metrics Evaluation

##### We have a dataset of 30 distinct “subjects” (objects vs. live subjects/pets), split into:
##### - **Real images**: stored under `data/<subject_name>/…`
##### - **Generated images**: stored under `results/{non-ppl,ppl}/<subject_name>/…`

##### We also have:
##### - `data/subjects.csv` with columns:
#####     - `subject_name` (matches each folder name)
#####    - `class`        (e.g. “dog”, “backpack”, etc.)
#####     - `live`         (boolean: True for pets, False for objects)
##### - `data/prompts.csv` with columns:
#####     - `prompt` (templates containing `{0}` → “sks”, `{1}` → the `class` value)
#####     - `live`   (boolean: whether this prompt applies to live subjects or objects)


##### **Evaluation protocol**:
##### - We generated up to **2** samples per prompt in `ppl` and also **2** for `non-ppl`.
##### - Metrics:
#####     1. **PRES** (avg pairwise DINO similarity real↔gen)
#####     2. **DIV**  (avg pairwise LPIPS distance among gen images)
#####     3. **CLIP-I** (avg cosine between CLIP image embeddings real↔gen)
#####     4. **CLIP-T** (avg cosine between CLIP text embeddings vs. gen images)


In [ ]:
import os
import pandas as pd
from metrics import clip_embeddings, div, pres

In [ ]:
subjects_df = pd.read_csv('../data/subjects.csv')
prompts_df  = pd.read_csv('../data/prompts.csv')

REAL_ROOT  = '../data'
GEN_ROOT  = '../results'
CONDITIONS = ['no_ppl', 'ppl']

#### Collecting CLIP-I, CLIP-T, PRES (between real and generated images)

In [ ]:
results = []
for cond in CONDITIONS:
    data_root = os.path.join(REAL_ROOT, cond)
    res_root = os.path.join(GEN_ROOT, cond)
    for _, row in subjects_df.iterrows():
        subject = row['subject_name']

        real_dir = os.path.join(data_root, subject)
        gen_dir  = os.path.join(res_root, subject)
        if not os.path.isdir(real_dir) or not os.path.isdir(gen_dir):
            continue 

        preservation   = pres.collect_pres(real_dir, gen_dir)
        clip_i, clip_t = None, None

        results.append({
            'condition': cond,
            'subject':   subject,
            'PRES':      preservation,
            'CLIP-I':    clip_i,
            'CLIP-T':    clip_t
        })

        print(f'{cond} {subject} {preservation:.4f}')

results_df = pd.DataFrame(results)

In [ ]:
results_df.to_csv("metric_results.csv", index=False)

In [ ]:
merged = (
    results_df
    .merge(subjects_df, left_on='subject', right_on='subject_name')
    .drop(columns=['subject_name']) 
)

pivot = (
    merged
    .pivot_table(
       index='class',
       columns='condition',
       values=['PRES','DIV','CLIP-I','CLIP-T'],
       aggfunc='mean'
    )
)
print(pivot)

pivot.to_csv("average_metric_results.csv", index=False)

#### Collecting DIV (between generated images of the same class)

In [ ]:
div_values = []
for cond in CONDITIONS:
    for _, row in subjects_df.iterrows():
        subject = row['subject_name']
        gen_dir = os.path.join(GEN_ROOT, cond, subject)
        if not os.path.isdir(gen_dir):
            print(f"Skipping {cond}/{subject}: directory not found")
            continue
        v = div.collect_div(gen_dir)
        print(f'{cond} {subject} {v:.4f}')
        div_values.append({'condition': cond, 'subject': subject, 'DIV': v})

div_df = pd.DataFrame(div_values)
div_df.to_csv("../results/div_results.csv", index=False)

div_df

In [ ]:
mean_div = (
    div_df
    .groupby('condition')['DIV']
    .mean()
    .reset_index(name='mean_DIV')
)
mean_div